In [1]:
import pandas as pd
import numpy as np

school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


school_data_complete.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [157]:
#District Summary

# Calculate the total number of schools
totalschools = school_data_complete['school_name'].nunique()

# Calculate the total number of students
num_students = school_data_complete['student_name'].count()

# Calculate the total budget
budget_sum = school_data['budget'].sum()

# Calculate the average math score 
avg_math = school_data_complete['math_score'].mean()

# Calculate the average reading score
avg_reading = school_data_complete['reading_score'].mean()

# Calculate the overall passing rate (overall average score), 
#i.e. (avg. math score + avg. reading score)/2
passing_rate = (avg_math + avg_reading)/2

# Calculate the percentage of students with a passing math score 
#(70 or greater)
passing_math = school_data_complete.loc[school_data_complete["math_score"] > 69, :]
num_mathpassing = passing_math['student_name'].count()
pass_mathrate = (num_mathpassing/num_students) * 100

# Calculate the percentage of students with a passing reading score 
#(70 or greater)
passing_read = school_data_complete.loc[school_data_complete["reading_score"] > 69, :]
num_readpassing = passing_read['student_name'].count()
pass_readrate = (num_readpassing/num_students) * 100

#overall pass rate
overpassrate = (pass_readrate + pass_mathrate) /2

# Create a dataframe to hold the above results
district_summary = pd.DataFrame({"Total Schools": [totalschools],
                                 "Total Students": [num_students],
                                 "Total Budget": [budget_sum],
                                 "Average Math Score": [avg_math],
                                 "Average Reading Score": [avg_reading],
                                 "% Passing Math": [pass_mathrate],
                                 "% Passing Reading": [pass_readrate],
                                 "% Overall Passing Rate": [overpassrate]
                                })

district_summary["Total Students"] = district_summary["Total Students"].map("{:,.0f}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.0f}".format)


district_summary



,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.985371,81.87784,74.980853,85.805463,80.393158


In [160]:
# School Summary
 
# Create an overview table that summarizes key metrics about each school,
#including:

groupby_school = school_data_complete.groupby("school_name")

# Total Students
ctstudents_sch = groupby_school['student_name'].count()

# Average Math Score
school_mathavg = groupby_school["math_score"].mean()
tl_schmathavg = pd.DataFrame(school_mathavg)

# Average Reading Score
school_readavg = groupby_school["reading_score"].mean()
tl_schreadavg = pd.DataFrame(school_readavg)

# % Passing Math
groupbyschool_math = passing_math.groupby("school_name")
num_mathpasssch =groupbyschool_math['student_name'].count()
pass_mathrate = (num_mathpasssch/ctstudents_sch) * 100
pass_mathrate_df = pd.DataFrame(pass_mathrate)

# % Passing Reading
groupbyschool_read = passing_read.groupby("school_name")
num_readpasssch =groupbyschool_read['student_name'].count()
pass_readrate = (num_readpasssch/ctstudents_sch) * 100
pass_readrate_df = pd.DataFrame(pass_readrate)

#merging % Passing Math and % Passing Reading
merge3 = pd.merge(pass_mathrate_df, pass_readrate_df, how="left", on="school_name")
merge3['% Overall Passing Rate'] = (merge3['student_name_x']+merge3['student_name_y'])/2


merge1 = pd.merge(tl_schmathavg, school_data, how="left", on="school_name")
merge2 = pd.merge(merge1, tl_schreadavg, how="left", on="school_name")
merge4 = pd.merge(merge2, merge3, how="left", on="school_name")

merge4['Per Student Budget'] = merge4['budget']/merge4['size']

merge4_rename = merge4.rename(columns={'type':'School Type',
                                       'school_name': ' ',
                                       'size': 'Total Students',
                                       'budget':'Total School Budget',
                                       'math_score':'Average Math Score',
                                       'reading_score':'Average Reading Score',   
                                       'student_name_x':'% Passing Math',
                                       'student_name_y':'% Passing Reading',
                                        })

merge4_rename = merge4_rename.set_index(' ')

merge4_reorg = merge4_rename[['School Type','Total Students', 
                               'Total School Budget', 'Per Student Budget', 'Average Math Score',
                               'Average Reading Score', '% Passing Math',
                               '% Passing Reading', '% Overall Passing Rate' ]]

merge4_reorg["Total School Budget"] = merge4_reorg["Total School Budget"].map("${:,.0f}".format)
merge4_reorg["Per Student Budget"] = merge4_reorg["Per Student Budget"].map("${:,.0f}".format)

merge4_reorg.head()


/Users/jessicaliroff/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jessicaliroff/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Bailey High School,District,4976,"$3,124,928",$628,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,"$1,081,356",$582,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,"$1,884,411",$639,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,"$1,763,916",$644,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,"$917,500",$625,83.351499,83.816757,93.392371,97.138965,95.265668


In [237]:
# Top Performing Schools (By Passing Rate)

mergereorg2 = merge4_rename[['School Type','Total Students', 
                               'Total School Budget', 'Per Student Budget', 'Average Math Score',
                               'Average Reading Score', '% Passing Math',
                               '% Passing Reading', '% Overall Passing Rate' ]]

sortedmerge_top = mergereorg2.sort_values(by="% Overall Passing Rate", ascending=False)

sortedmerge_top["Total School Budget"] = sortedmerge_top["Total School Budget"].map("${:,.0f}".format)
sortedmerge_top["Per Student Budget"] = sortedmerge_top["Per Student Budget"].map("${:,.0f}".format)


sortedmerge_top.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356",$582,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130",$638,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858",$609,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500",$625,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574",$578,83.274201,83.989488,93.867718,96.539641,95.203679


In [238]:
 #Bottom Performing Schools (By Passing Rate)
    
mergereorg3 = merge4_rename[['School Type','Total Students', 
                               'Total School Budget', 'Per Student Budget', 'Average Math Score',
                               'Average Reading Score', '% Passing Math',
                               '% Passing Reading', '% Overall Passing Rate' ]]

sortedmerge_btm = mergereorg2.sort_values(by="% Overall Passing Rate", ascending=True)

sortedmerge_btm["Total School Budget"] = sortedmerge_btm["Total School Budget"].map("${:,.0f}".format)
sortedmerge_btm["Per Student Budget"] = sortedmerge_btm["Per Student Budget"].map("${:,.0f}".format)

sortedmerge_btm.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363",$637,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411",$639,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635",$655,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650",$650,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916",$644,77.102592,80.746258,68.309602,79.299014,73.804308


In [203]:
#grouping by grade and HS

sc_9th = school_data_complete.loc[school_data_complete["grade"] == '9th', :]
sc_10th = school_data_complete.loc[school_data_complete["grade"] == '10th', :]
sc_11th = school_data_complete.loc[school_data_complete["grade"] == '11th', :]
sc_12th = school_data_complete.loc[school_data_complete["grade"] == '12th', :]
sc9th_groupbyhs = sc_9th.groupby('school_name')
sc10th_groupbyhs = sc_10th.groupby('school_name')
sc11th_groupbyhs = sc_9th.groupby('school_name')
sc12th_groupbyhs = sc_12th.groupby('school_name')


In [202]:
# Math Scores by Grade

#9th math score df
msc9th_avg = sc9th_groupbyhs['math_score'].mean()
msc9th_df = pd.DataFrame(msc9th_avg)
msc9th_df = msc9th_df.rename(columns={'math_score': '9th'})
msc9th_df

#10th math score df
msc10th_avg = sc10th_groupbyhs['math_score'].mean()
msc10th_df = pd.DataFrame(msc10th_avg)
msc10th_df = msc10th_df.rename(columns={'math_score': '10th'})
msc10th_df

#11th math score df
msc11th_avg = sc11th_groupbyhs['math_score'].mean()
msc11th_df = pd.DataFrame(msc11th_avg)
msc11th_df = msc11th_df.rename(columns={'math_score': '11th'})
msc11th_df

#12th math score df
msc12th_avg = sc12th_groupbyhs['math_score'].mean()
msc12th_df = pd.DataFrame(msc12th_avg)
msc12th_df = msc12th_df.rename(columns={'math_score': '12th'})
msc12th_df

combined_msc910 = pd.merge(msc9th_df, msc10th_df,how="left", on="school_name")
combined_msc1112 = pd.merge(msc11th_df, msc12th_df,how="left", on="school_name")
combined_mschs = pd.merge(combined_msc910, combined_msc1112,how="left", on="school_name")


combined_mschs.head()


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.083676,76.492218
Cabrera High School,83.094697,83.154506,83.094697,83.277487
Figueroa High School,76.403037,76.539974,76.403037,77.151369
Ford High School,77.361345,77.672316,77.361345,76.179963
Griffin High School,82.044010,84.229064,82.044010,83.356164


In [201]:
# Reading Score by Grade

#9th
rsc9th_avg = sc9th_groupbyhs['reading_score'].mean()
rsc9th_df = pd.DataFrame(rsc9th_avg)
rsc9th_df = rsc9th_df.rename(columns={'reading_score': '9th'})
rsc9th_df

#10th
rsc10th_avg = sc10th_groupbyhs['reading_score'].mean()
rsc10th_df = pd.DataFrame(rsc10th_avg)
rsc10th_df = rsc10th_df.rename(columns={'reading_score': '10th'})
rsc10th_df

#11th
rsc11th_avg = sc11th_groupbyhs['reading_score'].mean()
rsc11th_df = pd.DataFrame(rsc11th_avg)
rsc11th_df = rsc11th_df.rename(columns={'reading_score': '11th'})
rsc11th_df

#12th
rsc12th_avg = sc12th_groupbyhs['reading_score'].mean()
rsc12th_df = pd.DataFrame(rsc12th_avg)
rsc12th_df = rsc12th_df.rename(columns={'reading_score': '12th'})
rsc12th_df

combined_rsc910 = pd.merge(rsc9th_df, rsc10th_df,how="left", on="school_name")
combined_rsc1112 = pd.merge(rsc11th_df, rsc12th_df,how="left", on="school_name")
combined_rschs = pd.merge(combined_rsc910, combined_rsc1112,how="left", on="school_name")


combined_rschs.head()


,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,81.303155,80.912451
Cabrera High School,83.676136,84.253219,83.676136,84.287958
Figueroa High School,81.198598,81.408912,81.198598,81.384863
Ford High School,80.632653,81.262712,80.632653,80.662338
Griffin High School,83.369193,83.706897,83.369193,84.013699


In [231]:
# Scores by School Spending

# Create a table that breaks down school performances based on average Spending Ranges (Per Student). 
#Use 4 reasonable bins to group school spending. Include in the table each of the following:

# Average Math Score
# Average Reading Score
# % Passing Math
# % Passing Reading
# Overall Passing Rate (Average of the above two)

bins = [0, 585, 615, 645, 675]
group = ["<$585", "$585-615", "$615-645", "$645-675"]

mergereorg2["Spending Ranges (Per Student)"] = pd.cut(mergereorg2["Per Student Budget"],bins, labels=group)
averagespend = mergereorg2.groupby("Spending Ranges (Per Student)")

passmath = averagespend['% Passing Math'].mean()

passread = averagespend['% Passing Reading'].mean()

avgread_df = averagespend["Average Reading Score"].mean()

avg_spendrange = pd.DataFrame(averagespend["Average Math Score"].mean())

avg_spendrange['Average Reading Score'] = avgread_df

avg_spendrange['% Passing Math'] = passmath

avg_spendrange['% Passing Reading'] = passread

avg_spendrange['% Overall Passing Rate'] = (passmath + passread)/2

avg_spendrange


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


In [234]:
#Scores by School Size

bins = [0, 1000, 2000, 5000]
group = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

mergereorg2["School Size"] = pd.cut(mergereorg2["Total Students"],bins, labels=group)
averageschoolsize = mergereorg2.groupby("School Size")

passmath_hssz = averageschoolsize['% Passing Math'].mean()

passread_hssz = averageschoolsize['% Passing Reading'].mean()

avgread_df_hssz = averageschoolsize["Average Reading Score"].mean()

avg_hssz = pd.DataFrame(averageschoolsize["Average Math Score"].mean())

avg_hssz['Average Reading Score'] = avgread_df_hssz

avg_hssz['% Passing Math'] = passmath_hssz

avg_hssz['% Passing Reading'] = passread_hssz

avg_hssz['% Overall Passing Rate'] = (passmath_hssz + passread_hssz)/2

avg_hssz


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


In [236]:
#Scores by School Type

schooltype_groupby = mergereorg2.groupby("School Type")

passmath_hstype = schooltype_groupby['% Passing Math'].mean()

passread_hstype = schooltype_groupby['% Passing Reading'].mean()

avgread_df_hstype = schooltype_groupby["Average Reading Score"].mean()

avg_hstype = pd.DataFrame(schooltype_groupby["Average Math Score"].mean())

avg_hstype['Average Reading Score'] = avgread_df_hstype

avg_hstype['% Passing Math'] = passmath_hstype

avg_hstype['% Passing Reading'] = passread_hstype

avg_hstype['% Overall Passing Rate'] = (passmath_hstype + passread_hstype)/2

avg_hstype



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
